In [2]:
import requests
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
# API request to get product info for 1 specific product
url = "https://www.duluthtrading.com/api/ShopperProducts/getProducts?ids=76020&allImages=true&perPricebook=true"
response = requests.request("GET", url)
data = response.json()

In [4]:
# Data class to parse information from json/ dictionary tree
class TShirt:
    def __init__(self, product):
        self.id = product['productId']
        if 'name' in product:
            self.name = product['name']
        else:
            self.name = ""
        self.orderable = product['orderable']
        self.price = product['price']
        self.size = product['variationValues']['size']
        self.color = product['variationValues']['color']
        self.regprice = product['tieredPrices'][0]['price']
        
    def __str__(self):
        print()
        return f"""
                 Product ID: {self.id}
                 Name: {self.name}
                 Orderable: {self.orderable}
                 Price: {self.price}
                 Regular Price: {self.regprice}
                 Size: {self.size}
                 Color: {self.color}
        """
    
    def to_json(self):
        return {
                 "Id": self.id,
                 "Name": self.name,
                 "Orderable": self.orderable,
                 "Price": self.price,
                 "RegPrice": self.regprice,
                 "Size": self.size,
                 "Color": self.color
            }
                          

In [5]:
# Loop through all product variants and add to list
tshirt_list = []
for product_type in data['data']:
    url = product_type['slugUrl']
    for product in product_type['variants']:
        try:
            tshirt = TShirt(product).to_json()
            tshirt["link"] = url
            tshirt_list.append(tshirt)
        except:
            print(product)

In [8]:
# Convert from dictionary to dataframe
df = pd.DataFrame.from_dict(tshirt_list)

In [9]:
df

,Id,Name,Orderable,Price,RegPrice,Size,Color,link
0,000041013,Men's Longtail T Relaxed Fit Short Sleeve T-Sh...,True,24.50,24.50,MED,BLK,https://www.duluthtrading.com/mens-longtail-t-...
1,000041014,Men's Longtail T Relaxed Fit Short Sleeve T-Sh...,True,24.50,24.50,LRG,BLK,https://www.duluthtrading.com/mens-longtail-t-...
2,000041015,Men's Longtail T Relaxed Fit Short Sleeve T-Sh...,True,24.50,24.50,XLG,BLK,https://www.duluthtrading.com/mens-longtail-t-...
3,000041016,Men's Longtail T Relaxed Fit Short Sleeve T-Sh...,True,24.50,24.50,2XL,BLK,https://www.duluthtrading.com/mens-longtail-t-...
4,000041017,Men's Longtail T Relaxed Fit Short Sleeve T-Sh...,True,24.50,24.50,3XL,BLK,https://www.duluthtrading.com/mens-longtail-t-...
...,...,...,...,...,...,...,...,...
67,000547206,Men's Longtail T Relaxed Fit Short Sleeve T-Sh...,True,24.50,24.50,MED,SHN,https://www.duluthtrading.com/mens-longtail-t-...
68,000547207,Men's Longtail T Relaxed Fit Short Sleeve T-Sh...,True,24.50,24.50,SM,SHN,https://www.duluthtrading.com/mens-longtail-t-...
69,000547208,Men's Longtail T Relaxed Fit Short Sleeve T-Sh...,True,24.50,24.50,XLG,SHN,https://www.duluthtrading.com/mens-longtail-t-...
70,000547209,Men's Longtail T Relaxed Fit Short Sleeve T-Sh...,True,24.50,24.50,2XL,FTQ,https://www.duluthtrading.com/mens-longtail-t-...


In [10]:
# Add column to show the sale percentage
df['SalePercent'] = (1 - df['Price'] / df['RegPrice'])

In [13]:
# Filter to find the specific product/ sale price you want
sale_filter = df['SalePercent'] > 0
instock_filter = df['Orderable'] == True
size_filter = df['Size'] == 'MED'

df.loc[sale_filter & instock_filter & size_filter]

,Id,Name,Orderable,Price,RegPrice,Size,Color,link,SalePercent
35,000524136,Men's Longtail T Relaxed Fit Short Sleeve T-Sh...,True,14.99,24.50,MED,AMR,https://www.duluthtrading.com/mens-longtail-t-...,0.39


In [ ]:
# Optional: Email the list to your email address